In [1]:
!pip install -q efficientnet

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import tensorflow as tf
import supporting_functions as sf
import efficientnet.tfkeras as efn
import tensorflow.keras.layers as L

IMAGE_SIZE, BATCH_SIZE = sf.get_sizes()
EPOCHS = 3

print(IMAGE_SIZE)
print(BATCH_SIZE)
print(EPOCHS)

[768, 768]
128
3


In [3]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


In [10]:
# hardcoded cuz I'm a hack
NUM_TRAINING_IMAGES = 33126
NUM_TEST_IMAGES = 10982

TRAIN_STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
TEST_STEPS = NUM_TEST_IMAGES // BATCH_SIZE
print('Dataset: {} training images, {} test images'.format(NUM_TRAINING_IMAGES, NUM_TEST_IMAGES))

Dataset: 33126 training images, 10982 test images


In [5]:
def efiNet_model():
    model = efn.EfficientNetB5(input_shape=(*IMAGE_SIZE, 3), weights='imagenet', include_top=False)
    return model

In [6]:
# Defining the optimizer and compiling the model
with strategy.scope():
    model = tf.keras.Sequential([
        efiNet_model(),
        L.GlobalAveragePooling2D(),
        #L.Dense(1024, activation = 'relu'),
        #L.Dropout(0.3) , 
        L.Dense(512, activation= 'relu'), 
        L.Dropout(0.25), 
        #L.Dense(256, activation='relu'), 
        #L.Dropout(0.2), 
        #L.Dense(128, activation='relu'), 
        #L.Dropout(0.15), 
        L.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss = 'binary_crossentropy',
        metrics=[tf.keras.metrics.AUC()]
    )

115515392/115515256 [==============================] - 4s 0us/step


In [7]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b5 (Model)      (None, 24, 24, 2048)      28513520  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 29,563,121
Trainable params: 29,390,385
Non-trainable params: 172,736
_________________________________________________________________
None


In [8]:
train_dataset = sf.get_training_dataset()
print(train_dataset)

test_dataset = sf.get_testing_dataset()
print(test_dataset)

# Fitting the model
history = model.fit(
    train_dataset,
    steps_per_epoch=TRAIN_STEPS_PER_EPOCH,
    epochs=EPOCHS,
    use_multiprocessing=True
    #validation_data=get_validation_dataset(),
    #validation_steps=VAL_STEPS_PER_EPOCH
)

<PrefetchDataset shapes: ((None, 768, 768, 3), (None,)), types: (tf.float32, tf.int64)>
<PrefetchDataset shapes: ((None, 768, 768, 3), (None,)), types: (tf.float32, tf.int64)>
Epoch 1/3
258/258 [==============================] - 320s 1s/step - auc: 0.7486 - loss: 0.0893
Epoch 2/3
258/258 [==============================] - 322s 1s/step - auc: 0.8101 - loss: 0.0770
Epoch 3/3
258/258 [==============================] - 322s 1s/step - auc: 0.8370 - loss: 0.0747


In [11]:
test_predictions = model.predict(test_dataset, steps=TEST_STEPS, use_multiprocessing=True)
print(test_predictions)

InvalidArgumentError: {{function_node __inference_predict_function_215239}} Feature: target (data type: int64) is required but could not be found.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]
	 [[MultiDeviceIteratorGetNextFromShard]]
	 [[RemoteCall]]
	 [[IteratorGetNextAsOptional_2]]